In [1]:
import pandas as pd
from typing import List, Optional
from transformers import AutoConfig, AutoTokenizer, AutoModel
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms.base import LLM
from tqdm import tqdm

class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "GLM"
            
    def load_model(self, llm_device="gpu", model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True, device='cuda:5').half()

    def _call(self, prompt: str, history: List[str] = [], stop: Optional[List[str]] = None):
        response, _ = self.model.chat(
            self.tokenizer, prompt,
            history=history[-self.history_len:] if self.history_len > 0 else [],
            max_length=self.max_token, temperature=self.temperature,
            top_p=self.top_p
        )
        return response

# 设置模型路径并加载模型
modelpath = "/data1/dxw_data/llm/chatglm3-6b"
llm = GLM()
llm.load_model(model_name_or_path=modelpath)


2024-06-22 18:40:07.970688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 18:40:08.122434: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 18:40:08.803024: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-06-22 18:40:08.803105: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# 定义LangChain链
prompt = PromptTemplate(
    input_variables=["comment"],
    template="""请概括以下评论的主题：{comment}，最好用几个词语概括。"""
)

chain = LLMChain(llm=llm, prompt=prompt)

# 读取Excel文件
file_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data1/cleaned_comments_dianping_hotpot.xlsx'
df = pd.read_excel(file_path)

# 初始化保存结果的Excel文件路径
output_file_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data1/cleaned_comments_with_themes_label.xlsx'

# 定义起始索引
start_index = 0  # 可以根据需要更改起始索引

# 定义使用GLM模型对评论进行概括的函数
def summarize_comment(comment, index):
    try:
        summary = chain.run(comment)
        print(f"Processed comment {index + 1}: {summary}")
        return summary
    except Exception as e:
        print(f"Error processing comment {index + 1}: {e}")
        return f"Summarization error: {e}"

# 使用tqdm显示进度条，并估算剩余时间
for i in tqdm(range(start_index, len(df)), desc="Processing comments", unit="comment"):
    df.at[i, 'theme'] = summarize_comment(df.at[i, 'cleaned_comment'], i)
    df.to_excel(output_file_path, index=False)  # 保存结果到Excel文件
    print(f"Saved results up to comment {i + 1} to {output_file_path}")

print(f"Final results saved to {output_file_path}")

In [ ]:
# 根据提供的评论，我将提取并概括这些评论中的主题。以下是提取的主要主题：

# 1. **环境**：
#    - 干净、整洁、舒适
#    - 脏乱差
#    - 气氛热闹
#    - 风景优美

# 2. **服务**：
#    - 服务态度好、热情、友好
#    - 服务态度差、敷衍
#    - 服务员帮忙烤肉
#    - 免费续菜、小菜

# 3. **食物质量**：
#    - 食物新鲜
#    - 食物品质一般
#    - 烤肉味道好
#    - 分量不足

# 4. **价格**：
#    - 性价比高
#    - 价格较高
#    - 价格合理

# 5. **顾客体验**：
#    - 总体满意
#    - 总体不满意
#    - 排队等待时间长
#    - 推荐尝试

# 6. **特定菜品**：
#    - 小菜
#    - 烤肉
#    - 石锅拌饭
#    - 炒年糕
#    - 芝士鸡蛋



In [ ]:
# 定义LangChain链
prompt = PromptTemplate(
    input_variables=["comment"],
    template="""请概括以下评论的主题：{comment}，最好用几个词语概括。"""
)

chain = LLMChain(llm=llm, prompt=prompt)

# 读取Excel文件
file_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data2/cleaned_comments_dianping_barbecue.xlsx'
df = pd.read_excel(file_path)

# 初始化保存结果的Excel文件路径
output_file_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data2/cleaned_comments_with_themes_label.xlsx'

# 定义起始索引
start_index = 0  # 可以根据需要更改起始索引

# 定义使用GLM模型对评论进行概括的函数
def summarize_comment(comment, index):
    try:
        summary = chain.run(comment)
        print(f"Processed comment {index + 1}: {summary}")
        return summary
    except Exception as e:
        print(f"Error processing comment {index + 1}: {e}")
        return f"Summarization error: {e}"

# 使用tqdm显示进度条，并估算剩余时间
for i in tqdm(range(start_index, len(df)), desc="Processing comments", unit="comment"):
    df.at[i, 'theme'] = summarize_comment(df.at[i, 'cleaned_comment'], i)
    df.to_excel(output_file_path, index=False)  # 保存结果到Excel文件
    print(f"Saved results up to comment {i + 1} to {output_file_path}")

print(f"Final results saved to {output_file_path}")

In [ ]:
# 根据这些评论内容，可以将其分为以下几个主题：

# 1. **服务质量**
#    - 服务态度好或差
#    - 服务员的效率和热情
#    - 免费小菜续加服务

# 2. **环境**
#    - 环境优美或拥挤
#    - 干净与卫生
#    - 烤肉店的装修和氛围

# 3. **食物质量**
#    - 食材的新鲜度和口感
#    - 具体菜品（如烤肉、石锅拌饭、芝士玉米等）的味道评价
#    - 份量是否足够

# 4. **价格**
#    - 性价比高或低
#    - 价格合理性
#    - 套餐和单点菜品的价格评价

# 5. **用餐体验**
#    - 总体满意度
#    - 适合家庭聚餐、朋友聚会等场合
#    - 等待时间和排队情况

# 6. **推荐菜品**
#    - 特别推荐的菜品（如雪花牛肉、烤五花肉等）
#    - 对特定菜品的详细评价

# 7. **特殊场景**
#    - 节假日或周末用餐体验
#    - 特定活动或团购优惠


In [ ]:
# 定义LangChain链
prompt = PromptTemplate(
    input_variables=["comment"],
    template="""请概括以下评论的主题：{comment}，最好用几个词语概括。"""
)

chain = LLMChain(llm=llm, prompt=prompt)

# 读取Excel文件
file_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data3/cleaned_comments_dianping_cake.xlsx'
df = pd.read_excel(file_path)

# 初始化保存结果的Excel文件路径
output_file_path = '/data1/dxw_data/llm/RAG-mkt-kmeans/data3/cleaned_comments_with_themes_label.xlsx'

# 定义起始索引
start_index = 0  # 可以根据需要更改起始索引

# 定义使用GLM模型对评论进行概括的函数
def summarize_comment(comment, index):
    try:
        summary = chain.run(comment)
        print(f"Processed comment {index + 1}: {summary}")
        return summary
    except Exception as e:
        print(f"Error processing comment {index + 1}: {e}")
        return f"Summarization error: {e}"

# 使用tqdm显示进度条，并估算剩余时间
for i in tqdm(range(start_index, len(df)), desc="Processing comments", unit="comment"):
    df.at[i, 'theme'] = summarize_comment(df.at[i, 'cleaned_comment'], i)
    df.to_excel(output_file_path, index=False)  # 保存结果到Excel文件
    print(f"Saved results up to comment {i + 1} to {output_file_path}")

print(f"Final results saved to {output_file_path}")

In [ ]:
# 根据上述评论内容，可以将其分为以下几个主题：

# 1. **食物口感和质量**
#    - 芋泥蛋糕、爆浆巧克力等具体蛋糕的口感描述（松软、细腻、甜度适中）
#    - 特定口味的好评（芋泥、巧克力、原味等）
#    - 芝士、咸蛋黄等其他口味的评价

# 2. **价格和性价比**
#    - 价格高低评价
#    - 性价比高或低的意见
#    - 对价格合理性和是否值得购买的看法

# 3. **服务质量**
#    - 服务态度和效率的评价（店员态度、管理问题）
#    - 排队时间和购买体验（排队时间长、等待过程）

# 4. **环境**
#    - 店内环境描述和评价
#    - 排队环境和店铺位置

# 5. **购买体验**
#    - 排队购买经历和建议（什么时候排队比较好）
#    - 购买过程中的推荐和不推荐意见
#    - 特定时段的购买体验（下午、傍晚）

# 6. **综合评价**
#    - 整体满意度
#    - 推荐与否（是否值得一试、推荐给其他人）

# 7. **特定体验和情感**
#    - 个人情感和回忆（童年回忆、特别体验）
#    - 特殊场景的体验（雨天、周末、节假日）
